# Motivation
论文原文：[《Graph Attention Networks》](https://arxiv.org/abs/1710.10903)

参考学习地址：[向往的GAT(图注意力网络的原理、实现及计算复杂度)](https://zhuanlan.zhihu.com/p/81350196)

GCN擅长处理Transductive任务（训练和测试使用同样的图结构），而对于inductive任务（即处理动态图问题，通常训练只在子图上进行，测试需要处理未知顶点），以及有向图瓶颈（难以分配不同的学习权重给不同的neighbor）。GAT有两种运算方式：
* Global Graph Attention：每一个顶点$i$都对于图上任意顶点进行attention计算
* Mask Graph Attention：注意力机制的运算只在邻居顶点上进行（通常的选择）

<img style="display: block; margin: 0 auto;" src="../../../assets/images/gat.png" width = "600" height = "400" alt="GAT" align=center />

## 注意力系数（Attention Coefficient）
对于顶点$i$，逐个计算它的邻居们（$j\in \mathcal{N}_i$）和它自己之间的相似系数
$$
\begin{align}
e_{ij}=a([Wh_i||Wh_j]), \text{, where } j\in \mathcal{N}_i
\end{align}
$$
首先是用一个共享参数$W$的线性映射对于顶点的特征进行增维（常见的特征增强方式，feature augment），$[\cdot || \cdot ]$ 对于顶点$i$，$j$的变换后的特征进行concat，最后$a(\cdot)$把拼接后的高维映射到一个实数上。有了相关系数，归一化后即是注意力系数，这里使用的softmax（上图左）：
$$
\begin{align}
\alpha_{ij} = \frac{exp(LeakyReLU(e_{ij}))}{\sum_{k\in \mathcal{N}_i} exp(LeakyReLU(e_{ik}))}
\end{align}
$$

## 加权求和
根据计算好的注意力系数，把特征加权求和（aggregate）
$$
\begin{align}
h_i'=\sigma (\sum_{j\in \mathcal{N}_i} \alpha_{ij}Wh_j)
\end{align}
$$
其中，$h_i'$就是GAT输出的对于每个顶点$i$的新特征（融合了邻域信息），$\sigma (\cdot)$是激活函数。上式增加multi-head后（如图右）：
$$
\begin{align}
h_i'(K)=\|_{k=1}^K\sigma (\sum_{j\in \mathcal{N}_i} \alpha_{ij}^kW^kh_j)
\end{align}
$$

# 一些思考
* GCN和GAT都是将邻居顶点的特征聚合到中心顶点（aggregate），利用graph上的local stationary学习新的顶点特征表达。不同在于GCN使用拉普拉斯矩阵，GAT使用attention系数。
* GCN源于谱域（Spectral），而GAT属于空域（Vertex），运算方式是逐顶点计算（node-wise）。所以GAT更适合有向图
* GAT重要的学习参数是$W$和$a(\cdot)$，这两个参数仅与顶点特征相关，而与图结构无关，测试推理任务中改变图结构对于GAT影响不大，改变$\mathcal{N}_i$即可，所以更适用于inductive任务。相反GCN是全图计算，一次计算更新全图的节点特征，学习的参数与图结构相关

